# Machine Learning for Demand Forecasting

### Use case - predicting the demand for items at a libary

In [3]:
sfOptions = {
  "sfURL" : "datalytyx.east-us-2.azure.snowflakecomputing.com",
  "sfAccount" : "datalytyx",
  "sfUser" : "THOMASROWE",
  "sfPassword" : "Ketton123",
  "sfRole": "DATABRICKS",
  "sfDatabase" : "DATABRICKS_DEMO",
  "sfSchema" : "SEATTLE_LIBRARY",
  "sfWarehouse" : "DATASCIENCE_WH"
}
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

In [4]:
df = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
  .options(**sfOptions) \
  .option("query", """select * from library_collection_inventory where reportdate in ('2017-09-01T00:00:00','2017-10-01T00:00:00', '2017-11-01T00:00:00', '2017-12-01T00:00:00', '2018-01-01T00:00:00', '2018-01-01T00:00:00', '2018-02-01T00:00:00', '2018-02-01T00:00:00', '2018-03-01T00:00:00', '2018-04-01T00:00:00', '2018-05-01T00:00:00', '2018-06-01T00:00:00', '2018-07-01T00:00:00') limit 1000000""").load()

In [5]:
import pandas as pd 
import io
import csv
import re
import random
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


pd_data = df.toPandas()

def xstr(s):
    if s is None:
        return ''
    return str(s)

is_fiction = []
is_romance = []
is_crime = []
is_religious = []
is_sciencefiction = []
is_horror = []

subjects_c = []

for i in pd_data.loc[:,'SUBJECTS']:

  subjects_c.append(xstr(i))

for i in subjects_c:

    #df.loc[i,'Subjects'] = i.astype(str)
    m = re.search(r'Fiction', i,flags=re.IGNORECASE)
    k = re.search(r'Romance',i,flags=re.IGNORECASE)
    o = re.search(r'Crime',i,flags=re.IGNORECASE)
    p = re.search(r'religious',i,flags=re.IGNORECASE)
    q = re.search(r'ScienceFiction',i,flags=re.IGNORECASE)
    r = re.search(r'Horror',i,flags=re.IGNORECASE)

    if m is None:
        is_fiction.append(0)
    elif m is not None:
        is_fiction.append(1)

    if k is None:
        is_romance.append(0)

    elif k is not None:
        is_romance.append(1)

    if o is None:
        is_crime.append(0)

    elif o is not None:
        is_crime.append(1)

    if p is None:
        is_religious.append(0)

    elif p is not None:
        is_religious.append(1)

    if q is None:

        is_sciencefiction.append(0)

    elif q is not None:

         is_sciencefiction.append(0)

    if r is None:

        is_horror.append(0)
    elif r is not None:

        is_horror.append(1)

pd_data['is_romance'] = is_romance
pd_data['is_crime'] = is_crime
pd_data['is_religious'] = is_religious
pd_data['is_sciencefiction'] = is_sciencefiction
pd_data['is_horror'] = is_horror


ItemLocation = pd_data['ITEMLOCATION']

Itemlocation = pd.get_dummies(ItemLocation)


label_names = ['0','1']
labels = pd_data['is_romance']
feature_names = ['0','1']
features = Itemlocation

Itemlocation = ItemLocation


train, test = train_test_split(features, test_size=0.2)


train_labels, test_labels = train_test_split(labels, test_size=0.2)



train = np.array(train)
test = np.array(test)


gnb = GaussianNB()

train = train.reshape(-1, 1)
test = test.reshape(-1, 1)

chunkSize = 800000

for i in range(0, len(train), chunkSize):

    chunk = train[i:i+chunkSize]

    for i in range(13):

        model = gnb.fit(chunk, train_labels)

        preds = gnb.predict(test)

    #print(accuracy_score(test_labels,test[0:209715]))

















In [6]:
print(df)

DataFrame[BIBNUM: string, TITLE: string, AUTHOR: string, ISBN: string, PUBLICATIONYEAR: string, PUBLISHER: string, SUBJECTS: string, ITEMTYPE: string, ITEMCOLLECTION: string, FLOATINGITEM: string, ITEMLOCATION: string, REPORTDATE: string, ITEMCOUNT: string]

In [7]:
# Create a view or table

temp_table_name = "library_collection_inventory"

print(temp_table_name)

df.createOrReplaceTempView(temp_table_name)

library_collection_inventory

In [8]:
%sql
select * from library_collection_inventory limit 1000

BIBNUM,TITLE,AUTHOR,ISBN,PUBLICATIONYEAR,PUBLISHER,SUBJECTS,ITEMTYPE,ITEMCOLLECTION,FLOATINGITEM,ITEMLOCATION,REPORTDATE,ITEMCOUNT
504582,"Poems, 1963-1983 / C.K. Williams.","Williams, C. K. (Charles Kenneth), 1936-2015",0374235163,1988.,"Farrar Straus Giroux,",null,acbk,canf,NA,cen,2017-10-01T00:00:00,1
3199731,Paws off the pearl! / Geronimo Stilton ; [illustrations by Giuseppe Facciotto (design) and Alessandro Costa (color) ; graphics by Marta Lorini and Chiara Cebraro ; translated by Julia Heim].,"Stilton, Geronimo","1338032925, 9781338032925",[2016],"Scholastic Inc.,","Mice Juvenile fiction, Oysters Juvenile fiction, Humorous stories",jcbk,ccser,NA,cen,2017-10-01T00:00:00,2
3129683,"Spider-Gwen. [Vol. 0], Most wanted? / Jason Latour, writer ; Robbi Rodriguez, artist ; Rico Renzi, color artist ; VC's Clayton Cowles, letterer.","Latour, Jason, 1977-","0785197737, 9780785197737",[2015],"Marvel Worldwide, Inc., a subsidary of Marvel Entertainment, LLC,","Superheroes Comic books strips etc, Good and evil Comic books strips etc, Graphic novels, Comics Graphic works, Superhero comic books strips etc, Science fiction comic books strips etc, Comic books strips etc",acbk,nycomic,NA,col,2017-10-01T00:00:00,1
2861779,"Lời ru của mẹ / Mai Hương, sưu tầm và tuyển chọn.",null,8936046618706,2012.,"Văn Học,","Lullabies Vietnam Juvenile literature, Lullabies Juvenile literature",jcbk,ncln,NA,rbe,2017-10-01T00:00:00,1
3047307,The court-martial of Paul Revere : a son of liberty and America's forgotten military disaster / by Michael M. Greenburg.,"Greenburg, Michael M.","1611685354, 9781611685350",2014.,"ForeEdge,","Revere Paul 1735 1818, Penobscot Expedition 1779, Massachusetts History Revolution 1775 1783",acbk,canf,NA,cen,2017-10-01T00:00:00,1
894447,"""Main """"7810"""" police journal.""",null,null,1923-1926.,"Seattle Police Department,",Police Washington State Seattle Periodicals,arper,caref,NA,cen,2017-10-01T00:00:00,3
3175601,"Background checks for firearm transfers / U.S. Department of Justice, Office of Justice Programs, Bureau of Justice Statistics.",null,null,2000-,"U.S. Dept. of Justice, Bureau of Justice Statistics,","Gun control United States Statistics Periodicals, Firearms ownership Law and legislation United States Statistics Periodicals",arper,caref,NA,cen,2017-10-01T00:00:00,1
2051332,Spies' wives : stories of CIA families abroad / Karen L. Chiao and Mariellen B. O'Brien.,"Chiao, Karen L.",0887393217,c2001.,"Creative Arts Book Co.,","United States Central Intelligence Agency, Spies United States Spouses Biography, Spies United States Biography",acbk,canf,NA,cen,2017-10-01T00:00:00,1
2737761,Only forever / Linda Lael Miller.,"Miller, Linda Lael","037318493X, 9780373184934",c2011.,"Harlequin,","Television broadcasting Fiction, Seattle Wash Fiction, Ranchers Texas Fiction, Museum curators Fiction, Love stories",acbk,nafic,NA,lcy,2017-10-01T00:00:00,1
3004596,How to start a dragon academy / adapted by Erica David.,"David, Erica","1481419250, 1481419269, 9781481419253, 9781481419260",2014.,"Simon Spotlight,","Dragons Fiction, Vikings Fiction",jcbk,ncrdr,Floating,bea,2017-10-01T00:00:00,1


In [9]:
for i in temp_table_name[:12]:
   if i == 1:
      
      print("hello")
  
  
  
  

In [10]:
%sql
select count(*) from `library_collection_inventory`

count(1)
1000000


In [11]:
%sql

/* Query the created temp table in a SQL cell */

select BibNum, Title, ReportDate, SUM(ItemCount) AS ItemTotal from `library_collection_inventory`
WHERE BibNum = 1835141
GROUP BY BibNum, Title, ReportDate
ORDER BY ItemTotal DESC

BibNum,Title,ReportDate,ItemTotal


In [12]:
%sql
select BibNum, Title, ReportDate, ItemCount AS ItemTotal from `library_collection_inventory`
WHERE BibNum = 2969539
ORDER BY ItemTotal DESC

BibNum,Title,ReportDate,ItemTotal
2969539,"Doctor Who. Scream of the Shalka / BBCi ; written by Paul Cornell ; directed by Wilson Milam ; produced by Martin Trickey, James Goss and Mario Dubois.",2017-12-01T00:00:00,1
2969539,"Doctor Who. Scream of the Shalka / BBCi ; written by Paul Cornell ; directed by Wilson Milam ; produced by Martin Trickey, James Goss and Mario Dubois.",2017-12-01T00:00:00,1


In [13]:
%sql
select BibNum, Title, ReportDate, ItemCount AS ItemTotal from `library_collection_inventory`
WHERE BibNum = 2969539
ORDER BY ItemTotal DESC

BibNum,Title,ReportDate,ItemTotal
2969539,"Doctor Who. Scream of the Shalka / BBCi ; written by Paul Cornell ; directed by Wilson Milam ; produced by Martin Trickey, James Goss and Mario Dubois.",2017-12-01T00:00:00,1


In [14]:
%sql
select BibNum, Title, ITEMTYPE, ITEMCOLLECTION, ITEMLOCATION, ReportDate, SUM(ItemCount) AS ItemTotal 
from `library_collection_inventory`
GROUP BY BibNum, Title, ITEMTYPE, ITEMCOLLECTION, ITEMLOCATION, ReportDate
ORDER BY BibNum, ReportDate, Title;

BibNum,Title,ITEMTYPE,ITEMCOLLECTION,ITEMLOCATION,ReportDate,ItemTotal
100014,"We speak as liberators: young Black poets; an anthology, edited with an introd. by Orde Coombs.",acbk,naaanf,dth,2017-10-01T00:00:00,1.0
10002,Public water supply in Washington; an economic analysis of enabling laws for public water supply districts / [by C. Dirck Ditwiler.,arbk,cs7r,cen,2017-12-01T00:00:00,1.0
100022,"They also spoke; an essay on Negro literature in America, 1787-1930 / by Kenny J. Williams.",acbk,canf,cen,2018-01-01T00:00:00,1.0
100026,"A book of hours for Engelbert of Nassau, the Bodleian Library, Oxford. Introd. and legends by J. J. G. Alexander.",arbk,cs8r,cen,2017-10-01T00:00:00,1.0
100026,"A book of hours for Engelbert of Nassau, the Bodleian Library, Oxford. / Introd. and legends by J. J. G. Alexander.",arbk,cs8r,cen,2018-04-01T00:00:00,1.0
100027,The Catacombs and the Colosseum; the Roman Empire as the setting of primitive Christianity / [edited by] Stephen Benko [and] John J. O'Rourke.,acbk,canf,cen,2017-12-01T00:00:00,1.0
10003,The area of origin and a Columbia River diversion / by Ralph Johnson.,arbk,cs7r,cen,2017-12-01T00:00:00,1.0
100034,The third ear: a Black glossary / by Hermese E. Roberts.,arbk,naaar,dth,2017-12-01T00:00:00,1.0
1000364,The struggle for survival.,acbk,canf,cen,2018-06-01T00:00:00,1.0
100037,"The Premo camera, Rochester Optical Co., 1898; cameras, lenses, accessories, illustrated catalog and historical introduction.",arbk,cs8r,cen,2017-12-01T00:00:00,1.0


In [15]:
%sql
select BibNum, Title, ITEMTYPE, ITEMCOLLECTION, ITEMLOCATION, ReportDate, SUM(ItemCount) AS ItemTotal from `library_collection_inventory`
GROUP BY BibNum, Title, ITEMTYPE, ITEMCOLLECTION, ITEMLOCATION, ReportDate
ORDER BY BibNum, ReportDate, Title;

BibNum,Title,ITEMTYPE,ITEMCOLLECTION,ITEMLOCATION,ReportDate,ItemTotal
100011,Continuity and change: preservation in city planning / Alexander Papageorgiou. With a pref. by Frederick Gutheim. [Translated by Gerald Onn].,acbk,canf,cen,2017-12-01T00:00:00,1.0
100014,"We speak as liberators: young Black poets; an anthology, edited with an introd. by Orde Coombs.",acbk,naaanf,dth,2017-10-01T00:00:00,1.0
100014,We speak as liberators: young Black poets; an anthology / edited with an introd. by Orde Coombs.,acbk,naaanf,dth,2018-04-01T00:00:00,1.0
10002,Public water supply in Washington; an economic analysis of enabling laws for public water supply districts / [by C. Dirck Ditwiler.,arbk,cs7r,cen,2017-12-01T00:00:00,1.0
100022,"They also spoke; an essay on Negro literature in America, 1787-1930 / by Kenny J. Williams.",acbk,canf,cen,2018-01-01T00:00:00,1.0
100022,"They also spoke; an essay on Negro literature in America, 1787-1930 / by Kenny J. Williams.",acbk,canf,cen,2018-04-01T00:00:00,1.0
100026,"A book of hours for Engelbert of Nassau, the Bodleian Library, Oxford. / Introd. and legends by J. J. G. Alexander.",arbk,cs8r,cen,2018-04-01T00:00:00,1.0
100027,The Catacombs and the Colosseum; the Roman Empire as the setting of primitive Christianity / [edited by] Stephen Benko [and] John J. O'Rourke.,acbk,canf,cen,2017-12-01T00:00:00,1.0
10003,The area of origin and a Columbia River diversion / by Ralph Johnson.,arbk,cs7r,cen,2017-12-01T00:00:00,1.0
1000364,The struggle for survival.,acbk,canf,cen,2018-06-01T00:00:00,1.0


In [16]:
%sql
select distinct count(BibNum, Title, ReportDate, ItemCount)
from library_collection_inventory

"count(BibNum, Title, ReportDate, ItemCount)"
994672


In [17]:
item_counts = sql("select CAST(ItemCount AS INTEGER) AS ITEMCOUNT, count(*) as Frequency from library_collection_inventory GROUP BY ItemCount ORDER BY CAST(ItemCount AS INTEGER) asc")

In [18]:
display(item_counts.orderBy('ITEMCOUNT'))

ITEMCOUNT,Frequency
70,1
71,2
72,2
73,1
74,1
75,2
76,5
77,1
78,2
79,3


In [19]:
item_counts.write.format("net.snowflake.spark.snowflake").options(**sfOptions).option('dbtable', 'cust_lifetime_values').mode('overwrite').saveAsTable('cust_lifetime_values')